In [2]:

import numpy as np                   # advanced math library
import matplotlib.pyplot as plt 
import seaborn as sns     # MATLAB like plotting routines
import random                        # for generating random numbers
#
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
#
import os
#
import pandas as pd
import tensorflow as tf
#
from tensorflow.keras.models import Model
#
from tensorflow.keras.layers import Dense, Flatten, Input, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
#
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

from PIL import Image as im
#
import scipy.sparse
#
print(tf.__version__)

from keras.datasets import mnist     # MNIST dataset is included in Keras
from keras.models import Sequential  # Model type to be used


from keras.layers.core import Dense, Dropout, Activation # Types of layers to be used in our model
from keras.utils import np_utils      

     |████████████████████████████████| 588.3 MB 29 kB/s  eta 0:00:01    |██▌                             | 46.3 MB 325 kB/s eta 0:27:44     |██▋                             | 47.6 MB 40 kB/s eta 3:40:22     |██▋                             | 48.8 MB 1.4 MB/s eta 0:06:34     |█████▊                          | 104.3 MB 1.9 MB/s eta 0:04:14     |███████▏                        | 131.6 MB 1.8 MB/s eta 0:04:15     |█████████████████▉              | 326.9 MB 385 kB/s eta 0:11:18     |█████████████████▉              | 327.8 MB 576 kB/s eta 0:07:32     |██████████████████▋             | 341.6 MB 1.6 MB/s eta 0:02:37     |███████████████████████████▌    | 505.4 MB 2.0 MB/s eta 0:00:43     |█████████████████████████████   | 532.6 MB 1.8 MB/s eta 0:00:31ta 0:12:51
     |████████████████████████████████| 21.5 MB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 1.4 MB/s eta 0:00:01
     |██████████████████████████

2023-02-27 09:47:19.991843: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 09:47:20.159106: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-27 09:47:20.177174: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-27 09:47:20.177241: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

2.11.0


In [3]:
b = np.load('movie_recommend.npz')
print(b.files)

['format', 'shape', 'data', 'row', 'col']


In [4]:
print(b['data']) #data = 1000209

[5 5 5 ... 4 4 5]


In [5]:
# Carga el archivo .npz
archive_title_npz = np.load('movie_titles.npz')
archive_movie_npz= np.load('movie_recommend.npz')

# Claves disponibles en el archivo de títulos
print("Claves disponibles en el archivo de títulos:", archive_title_npz.files)

# claves disponibles en el archivo de recomendaciones de películas
print("Claves disponibles en el archivo de recomendaciones de películas:", archive_movie_npz.files)

# Obtiene el array de títulos
title_array = archive_title_npz['titles']

Claves disponibles en el archivo de títulos: ['titles']
Claves disponibles en el archivo de recomendaciones de películas: ['format', 'shape', 'data', 'row', 'col']


In [6]:
len(title_array)

3706

In [7]:
sparse_matrix = scipy.sparse.load_npz('movie_recommend.npz')

In [8]:
sparse_matrix #6040 users and 3706 movies

<6040x3706 sparse matrix of type '<class 'numpy.int8'>'
	with 1000209 stored elements in COOrdinate format>

In [9]:
sparse_matrix = sparse_matrix.toarray()

In [10]:
def recommend_system(sdata):
    sdata = sdata.astype(np.float64)
    tolerance = 0.9
    u, s,v = np.linalg.svd(sdata, full_matrices=False)
    s = np.diag(s)
    to_cut = int(tolerance*np.shape(s)[0])
    #print(np.shape(u),np.shape(s),np.shape(v))
    u = u[:,:-to_cut]
    s = s[:-to_cut,:-to_cut]
    v = v[:-to_cut,:]
    #print(np.shape(u),np.shape(s),np.shape(v))
    a = u @ s @ v
    return a

In [11]:

def recommend_me(users): #I tried to vectorize it but for some reason it changed the suggetions, so 
    #I guess I screw it up in some part of this cell
    system = recommend_system(sparse_matrix)
    scores = np.zeros((len(users),np.shape(system)[1]))
    known_taste = sparse_matrix[users,:]
    not_seen_movies = (known_taste==0) #get which movies the users have not seen
    scores = np.multiply(not_seen_movies,system[users])
    print(np.shape(scores))
    top_ind = np.argpartition(scores, -3,axis=1)[:,-3:] #get top 3 suggestions
    max_score = np.max(scores,axis=1)
    print(max_score)
    for user in np.arange(0,np.shape(top_ind)[0]): #double for just for printing
        for movie_ind in np.arange(0,np.shape(top_ind)[1]):
             movie = top_ind[user,movie_ind]
             percentage = "{0:.0%}".format(scores[user,movie]/max_score[user])
             print('User', users[user], ', we are', percentage, 'sure you will like this movie:',title_array[movie])
        print('\n')
    return None

In [12]:
recommend_me([450,1222,4989])

(3, 3706)
[3.11062513 2.36699105 1.84073247]
User 450 , we are 81% sure you will like this movie: American Beauty (1999)
User 450 , we are 98% sure you will like this movie: Toy Story (1995)
User 450 , we are 100% sure you will like this movie: Akira (1988)


User 1222 , we are 78% sure you will like this movie: Pretty Woman (1990)
User 1222 , we are 79% sure you will like this movie: Batman (1989)
User 1222 , we are 100% sure you will like this movie: Doctor Dolittle (1967)


User 4989 , we are 93% sure you will like this movie: Gremlins (1984)
User 4989 , we are 100% sure you will like this movie: Wayne's World 2 (1993)
User 4989 , we are 94% sure you will like this movie: Saving Private Ryan (1998)


